Код отсюда: https://www.kaggle.com/alexryzhkov/lightautoml-baseline-tps-june-2021

Туториал по lightautoml https://lightautoml.readthedocs.io/en/latest/tutorials/tutor_2.html

In [1]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c tabular-playground-series-jun-2021
!pip install catboost
!pip install eli5
!pip install shap
!unzip /content/test.csv.zip
!unzip /content/train.csv.zip
!unzip /content/sample_submission.csv.zip

kaggle.json
  0% 0.00/3.14M [00:00<?, ?B/s]
100% 3.14M/3.14M [00:00<00:00, 105MB/s]
  0% 0.00/6.49M [00:00<?, ?B/s]
100% 6.49M/6.49M [00:00<00:00, 107MB/s]
  0% 0.00/264k [00:00<?, ?B/s]
100% 264k/264k [00:00<00:00, 84.9MB/s]
     |████████████████████████████████| 67.3MB 48kB/s 
     |████████████████████████████████| 112kB 28.5MB/s 
     |████████████████████████████████| 358kB 20.7MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491617 sha256=170015f47e708ae08ce788c605b317fcbfbe4b72942354ba443643b056adc3a2
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap
Archive:  /content/test.csv.zip
  inflating: test.csv                
Archive:  /content/train.csv.zip
  inflating: train.csv               
Archive:  /content/sample_submission.csv.zip
  inflating: sample_submission.csv   


In [2]:
!pip install -U lightautoml

     |████████████████████████████████| 256kB 31.6MB/s 
     |████████████████████████████████| 1.2MB 30.2MB/s 
     |████████████████████████████████| 296kB 47.4MB/s 
     |████████████████████████████████| 2.3MB 49.0MB/s 
     |████████████████████████████████| 430kB 46.0MB/s 
     |████████████████████████████████| 204kB 54.6MB/s 
     |████████████████████████████████| 235kB 43.3MB/s 
     |████████████████████████████████| 102kB 13.2MB/s 
     |████████████████████████████████| 174kB 51.3MB/s 
     |████████████████████████████████| 81kB 11.6MB/s 
     |████████████████████████████████| 901kB 39.6MB/s 
     |████████████████████████████████| 3.3MB 41.0MB/s 
     |████████████████████████████████| 9.2MB 39.6MB/s 
     |████████████████████████████████| 38.2MB 68kB/s 
     |████████████████████████████████| 81kB 12.3MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 112kB 59.8MB/s 
     |████████████████████████████████| 143kB 59.5MB/

# 1. Import necessary libraries

In [3]:
# Standard python libraries
import os
import time
import re

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

# 2. Parameters

In [4]:
N_THREADS = 256 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 1 * 3600 # Time in seconds for automl run
TARGET_NAME = 'target'

# 3. Data load

In [5]:
train_data = pd.read_csv('train.csv')
train_data[TARGET_NAME] = train_data[TARGET_NAME].str.slice(start=6).astype(int) - 1
train_data.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,0,6,1,0,0,0,0,7,0,0,0,3,0,1,0,0,3,3,1,0,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,11,0,0,0,0,0,9,0,0,0,0,0,0,3,0,1,3,0,0,0,0,0,0,0,1,1,0,0,3,0,0,0,0,0,0,2,0,0,5
1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,5
2,2,0,0,0,0,0,1,0,3,0,0,1,0,0,0,0,0,0,0,0,5,4,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,5,0,5,0,1,0,0,0,1,0,0,1,0,7,0,1,1,0,3,4,0,0,1,3,0,2,0,0,8,0,0,0,0,1,0,0,0,0,1
3,3,0,0,7,0,1,5,2,2,0,1,2,0,5,0,0,4,0,0,22,2,1,0,0,0,0,3,0,37,0,1,3,13,0,10,0,3,1,1,0,7,0,0,2,0,1,0,0,0,0,0,0,10,0,0,25,1,0,1,2,0,2,0,7,0,0,0,0,4,0,2,2,0,4,3,0,7
4,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [11]:
test_data = pd.read_csv('test.csv')
test_data.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,200000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,200001,1,2,0,0,0,0,0,0,0,0,1,0,2,2,0,0,1,0,0,0,2,0,0,1,0,3,0,1,0,0,2,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,3,2,0,0,0,0,0,0,1,7,0,1,3,1,3,0,0,0,0,3,0,0
2,200002,0,1,7,1,0,0,0,0,6,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,2,0,0,4,0,0,0,0,0,0,0,0,1,0,0,4,0,0,1,0,0,0,6,0,0,2,10,0,0,0,2,1,0,0,0,0,0,3,0,0,0,0,3,0,2,0,0
3,200003,0,0,0,4,3,1,0,0,0,0,7,0,0,0,7,0,0,0,5,0,1,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,1,0,0,7,5,0,0,0,0,0,0,0,0,0,2,3,0,0,0,1,0,0,0,0,0,4,0,0,0,1,0,0,0,4,0,0
4,200004,0,0,5,0,0,0,0,0,0,8,1,0,2,0,0,0,0,0,9,1,0,0,0,0,0,0,0,0,3,0,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0


In [6]:
submission = pd.read_csv('sample_submission.csv')
submission.head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,200000,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
1,200001,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
2,200002,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
3,200003,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
4,200004,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111


# 4. AutoML

## 4.1. Create Task

In [7]:
%%time

task = Task('multiclass',)

CPU times: user 6.37 ms, sys: 63 µs, total: 6.44 ms
Wall time: 9.64 ms


## 4.2. Setup columns roles

In [8]:
%%time

roles = {
    'target': TARGET_NAME,
    'drop': ['id'],
}

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.63 µs


## 4.3. Train on full data

In [9]:
%%time 

automl = TabularUtilizedAutoML(task = task, 
                               timeout = TIMEOUT,
                               cpu_limit = N_THREADS,
                               general_params = {'use_algos': [['lgb', 'cb', 'cb_tuned'], ['linear_l2', 'cb']]},
                               reader_params = {'n_jobs': N_THREADS},
                               )
oof_pred = automl.fit_predict(train_data, roles = roles)
print('oof_pred:\n{}\nShape = {}'.format(oof_pred[:10], oof_pred.shape))

Current random state: {'reader_params': {'random_state': 42}, 'general_params': {'return_all_predictions': False}}
Found reader_params in kwargs, need to combine
Merged variant for reader_params = {'n_jobs': 256, 'random_state': 42}
Found general_params in kwargs, need to combine
Merged variant for general_params = {'use_algos': [['lgb', 'cb', 'cb_tuned'], ['linear_l2', 'cb']], 'return_all_predictions': False}
Start automl preset with listed constraints:
- time: 3599.9851536750793 seconds
- cpus: 256 cores
- memory: 16 gb

Train data shape: (200000, 77)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 3548.8636877536774 secs
Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 1.75893
[200]	valid's multi_logloss: 1.75424
[300]	valid's multi_logloss: 1.7555
Early stopping, best iteratio

Time limit exceeded after calculating fold 1


Lvl_0_Pipe_0_Mod_0_LightGBM fitting and predicting completed
Start fitting Lvl_0_Pipe_0_Mod_1_CatBoost ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_CatBoost =====

0:	learn: 2.1728049	test: 2.1730550	best: 2.1730550 (0)	total: 412ms	remaining: 27m 27s
100:	learn: 1.7608880	test: 1.7687804	best: 1.7687804 (100)	total: 36s	remaining: 23m 10s
200:	learn: 1.7444941	test: 1.7577653	best: 1.7577653 (200)	total: 1m 7s	remaining: 21m 12s
300:	learn: 1.7369694	test: 1.7553245	best: 1.7553245 (300)	total: 1m 35s	remaining: 19m 33s
400:	learn: 1.7308842	test: 1.7540464	best: 1.7540464 (400)	total: 2m 2s	remaining: 18m 20s
500:	learn: 1.7244361	test: 1.7528771	best: 1.7528771 (500)	total: 2m 31s	remaining: 17m 37s
600:	learn: 1.7185919	test: 1.7523251	best: 1.7523251 (600)	total: 2m 58s	remaining: 16m 47s
700:	learn: 1.7131240	test: 1.7520251	best: 1.7520106 (699)	total: 3m 24s	remaining: 16m
800:	learn: 1.7080102	test: 1.7518689	best: 1.7518462 (797)	total: 3m 50s	remaining: 15m 19

Time limit exceeded after calculating fold 1


Lvl_0_Pipe_0_Mod_1_CatBoost fitting and predicting completed
Optuna may run 1 secs


...

Start fitting Lvl_0_Pipe_0_Mod_2_CatBoost ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_2_CatBoost =====

0:	learn: 2.1729409	test: 2.1731485	best: 2.1731485 (0)	total: 320ms	remaining: 21m 18s
100:	learn: 1.7669339	test: 1.7726229	best: 1.7726229 (100)	total: 31.2s	remaining: 20m 5s
200:	learn: 1.7511661	test: 1.7597123	best: 1.7597123 (200)	total: 1m	remaining: 18m 54s
300:	learn: 1.7457115	test: 1.7565433	best: 1.7565433 (300)	total: 1m 26s	remaining: 17m 46s
400:	learn: 1.7420037	test: 1.7548603	best: 1.7548603 (400)	total: 1m 52s	remaining: 16m 46s
500:	learn: 1.7382577	test: 1.7536908	best: 1.7536908 (500)	total: 2m 19s	remaining: 16m 12s
600:	learn: 1.7352083	test: 1.7530468	best: 1.7530455 (598)	total: 2m 44s	remaining: 15m 28s
700:	learn: 1.7325366	test: 1.7525887	best: 1.7525887 (700)	total: 3m 8s	remaining: 14m 46s
800:	learn: 1.7301623	test: 1.7522229	best: 1.7522229 (800)	total: 3m 32s	remaining: 14m 7s
900:	learn: 1.7279965	test: 1.7520129	best: 1.7520129 (900

Time limit exceeded after calculating fold 3


Lvl_0_Pipe_0_Mod_2_CatBoost fitting and predicting completed
Time left 682.637748003006


Time limit exceeded in one of the tasks. AutoML will blend level 1 models.                                         
Try to set higher time limits or use Profiler to find bottleneck and optimize Pipelines settings


Blending: Optimization starts with equal weights and score -1.7470666382503928
Blending, iter 0: score = -1.74706256334919, weights = [0.30171794 0.35706687 0.3412152 ]
Blending, iter 1: score = -1.7470625537043438, weights = [0.30116066 0.35948393 0.33935544]
Blending, iter 2: score = -1.7470625537043438, weights = [0.30116066 0.35948393 0.33935544]
No score update. Terminated

Automl preset training completed in 2923.35 seconds.
oof_pred:
array([[0.01812763, 0.01758972, 0.01590177, 0.00772718, 0.00409396,
        0.7589494 , 0.02983649, 0.10947287, 0.03830104],
       [0.03740199, 0.13258949, 0.07632411, 0.02374497, 0.00937621,
        0.46721923, 0.03390833, 0.11283244, 0.10660326],
       [0.05183011, 0.11258177, 0.08019474, 0.02383038, 0.01906432,
        0.15213107, 0.09430991, 0.28979713, 0.17626062],
       [0.05311953, 0.10709359, 0.10309666, 0.02954577, 0.03115356,
        0.10480604, 0.09692325, 0.29662645, 0.1776352 ],
       [0.060084  , 0.3083984 , 0.14751643, 0.0266003 ,

## 4.4. Predict for test data and check OOF score¶

In [12]:
%%time

test_pred = automl.predict(test_data)
print('Prediction for test data:\n{}\nShape = {}'.format(test_pred[:10], test_pred.shape))

print('Check scores...')
print('OOF score: {}'.format(log_loss(train_data[TARGET_NAME].values, oof_pred.data)))

Prediction for test data:
array([[0.05348641, 0.3672707 , 0.15677106, 0.02638205, 0.01312626,
        0.17018145, 0.02689078, 0.06281381, 0.12307748],
       [0.04164853, 0.08106397, 0.06216176, 0.01925035, 0.01376328,
        0.25311622, 0.07967722, 0.31903496, 0.13028376],
       [0.02117894, 0.02978867, 0.02309041, 0.01007608, 0.00621405,
        0.68225205, 0.03365135, 0.13306244, 0.06068605],
       [0.04800578, 0.118524  , 0.08143333, 0.04029392, 0.01779301,
        0.22230107, 0.07258399, 0.23729566, 0.16176929],
       [0.04223086, 0.11607187, 0.08137749, 0.02800404, 0.01443975,
        0.2972453 , 0.06323846, 0.21323314, 0.14415911],
       [0.0527746 , 0.20720667, 0.10607398, 0.03051032, 0.01192024,
        0.27228388, 0.04556207, 0.13641807, 0.13725021],
       [0.03755406, 0.11369773, 0.08230691, 0.03093368, 0.01827616,
        0.20262474, 0.08814791, 0.26282343, 0.16363543],
       [0.04694959, 0.4054562 , 0.16104674, 0.02763849, 0.0169553 ,
        0.05356741, 0.0409603 ,

ValueError: ignored

## 4.5. Prepare submission

In [13]:
submission.iloc[:, 1:] = test_pred.data
submission.to_csv('KTJune_lightautoml_2lvl_1hour.csv', index = False)